## CMPE 255 Assignments

## Frequent Pattern Mining Algorithms

 



In [47]:
# Import libraries
from collections import defaultdict
from csv import reader
from itertools import chain, combinations

In [48]:
# Given the following list of items
items = [
    ["maggi", "tea", "biscuit"],
    ["bread", "tea", "bournvita"],
    ["maggi", "tea", "cornflakes"],
    ["maggi", "bread", "tea", "biscuit"],
    ["jam", "maggi", "tea"],
    ["milk", "bournvita", "biscuit"],
    ["milk", "biscuit", "cornflakes"],
    ["bread", "tea", "bournvita"],
    ["jam", "maggi", "bread", "milk"],
    ["bread", "milk"],
    ["coffee", "cock", "biscuit", "cornflakes"],
    ["coffee", "cock", "biscuit", "cornflakes"],
    ["coffee", "suger", "bournvita"],
    ["bread", "coffee", "cock"],
    ["bread", "suger", "biscuit"],
    ["coffee", "suger", "cornflakes"],
    ["bread", "suger", "bournvita"],
    ["coffee", "suger"],
    ["bread", "coffee", "suger"],
    ["tea", "milk", "coffee", "cornflakes"],
]

## Apriori Algorithm

Apriori is the Algorithm used for finding frequent itemsets in a dataset for boolean association rule. This algorithm identifies the underlying relations between different items.

In [49]:
def powerset(s):
    return chain.from_iterable(combinations(s, r) for r in range(1, len(s)))


def getAboveMinSup(itemSet, items, minSup, globalItemSetWithSup):
    freqItemSet = set()
    localItemSetWithSup = defaultdict(int)

    for item in itemSet:
        for itemSet in items:
            if item.issubset(itemSet):
                globalItemSetWithSup[item] += 1
                localItemSetWithSup[item] += 1

    for item, supCount in localItemSetWithSup.items():
        support = float(supCount / len(items))
        if support >= minSup:
            freqItemSet.add(item)

    return freqItemSet


def getUnion(itemSet, length):
    return set(
        [i.union(j) for i in itemSet for j in itemSet if len(i.union(j)) == length]
    )


def pruning(candidateSet, prevFreqSet, length):
    tempCandidateSet = candidateSet.copy()
    for item in candidateSet:
        subsets = combinations(item, length)
        for subset in subsets:
            # if the subset is not in previous K-frequent get, then remove the set
            if frozenset(subset) not in prevFreqSet:
                tempCandidateSet.remove(item)
                break
    return tempCandidateSet


def associationRule(freqItemSet, itemSetWithSup, minConf):
    rules = []
    for k, itemSet in freqItemSet.items():
        for item in itemSet:
            subsets = powerset(item)
            for s in subsets:
                confidence = float(itemSetWithSup[item] / itemSetWithSup[frozenset(s)])
                if confidence > minConf:
                    rules.append([set(s), set(item.difference(s)), confidence])
    return rules


def getItemSetFromList(items):
    tempItemSet = set()

    for itemSet in items:
        for item in itemSet:
            tempItemSet.add(frozenset([item]))

    return tempItemSet


def apriori(items, minSup, minConf):
    C1ItemSet = getItemSetFromList(items)
    # Final result global frequent itemset
    globalFreqItemSet = dict()
    # Storing global itemset with support count
    globalItemSetWithSup = defaultdict(int)

    L1ItemSet = getAboveMinSup(C1ItemSet, items, minSup, globalItemSetWithSup)
    currentLSet = L1ItemSet
    k = 2

    # Calculating frequent item set
    while currentLSet:
        # Storing frequent itemset
        globalFreqItemSet[k - 1] = currentLSet
        # Self-joining Lk
        candidateSet = getUnion(currentLSet, k)
        # Perform subset testing and remove pruned supersets
        candidateSet = pruning(candidateSet, currentLSet, k - 1)
        # Scanning itemSet for counting support
        currentLSet = getAboveMinSup(
            candidateSet, items, minSup, globalItemSetWithSup
        )
        k += 1

    rules = associationRule(globalFreqItemSet, globalItemSetWithSup, minConf)
    rules.sort(key=lambda x: x[2])

    return globalFreqItemSet, rules

In [50]:
freqItemSet, rules = apriori(items, minSup=0.2, minConf=0.2)
print("Apriori Frequent item Set")
print(freqItemSet)
print()
print("Apriori Rules")
print(rules)

Apriori Frequent item Set
{1: {frozenset({'maggi'}), frozenset({'suger'}), frozenset({'bournvita'}), frozenset({'cornflakes'}), frozenset({'bread'}), frozenset({'biscuit'}), frozenset({'tea'}), frozenset({'milk'}), frozenset({'coffee'})}, 2: {frozenset({'cornflakes', 'coffee'}), frozenset({'suger', 'coffee'}), frozenset({'tea', 'maggi'})}}

Apriori Rules
[[{'coffee'}, {'cornflakes'}, 0.5], [{'coffee'}, {'suger'}, 0.5], [{'tea'}, {'maggi'}, 0.5714285714285714], [{'cornflakes'}, {'coffee'}, 0.6666666666666666], [{'suger'}, {'coffee'}, 0.6666666666666666], [{'maggi'}, {'tea'}, 0.8]]


# **FP Growth Algorithm**

This algorithm is used to find the frequent itemsets in a dataset. This algorithm is faster than the Apriori algorithm. With this technique we can find the product combination that are often bought together.

In [51]:
class Node:
    def __init__(self, itemName, frequency, parentNode):
        self.itemName = itemName
        self.count = frequency
        self.parent = parentNode
        self.children = {}
        self.next = None

    def increment(self, frequency):
        self.count += frequency

    def display(self, ind=1):
        print("  " * ind, self.itemName, " ", self.count)
        for child in list(self.children.values()):
            child.display(ind + 1)


def constT(items, frequency, minSup):
    headerTable = defaultdict(int)
    # Counting frequency and create header table
    for idx, itemSet in enumerate(items):
        for item in itemSet:
            headerTable[item] += frequency[idx]

    # Deleting items below minSup
    headerTable = dict(
        (item, sup) for item, sup in headerTable.items() if sup >= minSup
    )
    if len(headerTable) == 0:
        return None, None

    # HeaderTable column [Item: [frequency, headNode]]
    for item in headerTable:
        headerTable[item] = [headerTable[item], None]

    # Init Null head node
    fpTree = Node("Null", 1, None)
    # Update FP tree for each cleaned and sorted itemSet
    for idx, itemSet in enumerate(items):
        itemSet = [item for item in itemSet if item in headerTable]
        itemSet.sort(key=lambda item: headerTable[item][0], reverse=True)
        # Traverse from root to leaf, update tree with given item
        currentNode = fpTree
        for item in itemSet:
            currentNode = updateTree(item, currentNode, headerTable, frequency[idx])

    return fpTree, headerTable


def updateTree(item, treeNode, headerTable, frequency):
    if item in treeNode.children:
        # If the item already exists, increment the count
        treeNode.children[item].increment(frequency)
    else:
        # Create a new branch
        newItemNode = Node(item, frequency, treeNode)
        treeNode.children[item] = newItemNode
        # Link the new branch to header table
        updateHeaderTable(item, newItemNode, headerTable)

    return treeNode.children[item]


def updateHeaderTable(item, targetNode, headerTable):
    if headerTable[item][1] == None:
        headerTable[item][1] = targetNode
    else:
        currentNode = headerTable[item][1]
        # Traverse to the last node then link it to the target
        while currentNode.next != None:
            currentNode = currentNode.next
        currentNode.next = targetNode


def mineTree(headerTable, minSup, preFix, freqItemList):
    # Sort the items with frequency and create a list
    sortedItems = [
        item[0] for item in sorted(list(headerTable.items()), key=lambda p: p[1][0])
    ]
    # Start with the lowest frequency
    for item in sortedItems:
        # Pattern growth is achieved by the concatenation of suffix pattern with frequent patterns generated from conditional FP-tree
        newFreqSet = preFix.copy()
        newFreqSet.add(item)
        freqItemList.append(newFreqSet)
        # Find all prefix path, constrcut conditional pattern base
        conditionalPattBase, frequency = findPrefixPath(item, headerTable)
        # Construct conditonal FP Tree with conditional pattern base
        conditionalTree, newHeaderTable = constT(
            conditionalPattBase, frequency, minSup
        )
        if newHeaderTable != None:
            # Mining recursively on the tree
            mineTree(newHeaderTable, minSup, newFreqSet, freqItemList)


def findPrefixPath(basePat, headerTable):
    # First node in linked list
    treeNode = headerTable[basePat][1]
    condPats = []
    frequency = []
    while treeNode != None:
        prefixPath = []
        # From leaf node all the way to root
        ascendFPtree(treeNode, prefixPath)
        if len(prefixPath) > 1:
            # Storing the prefix path and it's corresponding count
            condPats.append(prefixPath[1:])
            frequency.append(treeNode.count)

        # Go to next node
        treeNode = treeNode.next
    return condPats, frequency


def ascendFPtree(node, prefixPath):
    if node.parent != None:
        prefixPath.append(node.itemName)
        ascendFPtree(node.parent, prefixPath)


def getFrequencyFromList(items):
    frequency = [1 for i in range(len(items))]
    return frequency


def associationRuleFPGrowth(freqItemSet, items, minConf):
    rules = []
    for itemSet in freqItemSet:
        subsets = powerset(itemSet)
        itemSetSup = getSupport(itemSet, items)
        for s in subsets:
            confidence = float(itemSetSup / getSupport(s, items))
            if confidence > minConf:
                rules.append([set(s), set(itemSet.difference(s)), confidence])
    return rules


def getSupport(testSet, items):
    count = 0
    for itemSet in items:
        if set(testSet).issubset(itemSet):
            count += 1
    return count


def fpgrowth(items, minSupRatio, minConf):
    frequency = getFrequencyFromList(items)
    minSup = len(items) * minSupRatio
    fpTree, headerTable = constT(items, frequency, minSup)
    if fpTree == None:
        print("No frequent item set")
    else:
        freqItems = []
        mineTree(headerTable, minSup, set(), freqItems)
        rules = associationRuleFPGrowth(freqItems, items, minConf)
        return freqItems, rules

In [52]:
freqItems, rules = fpgrowth(items, minSupRatio=0.2, minConf=0.2)
# Frequent items lists, Below observation shows that the items coffee and Cornflakes are purchased together. And Sugar and Coffee are purchased together etc..
print("FP growth Frequent Items lists")
print(freqItems)
print()


print("FP Rules")
print(rules)

FP growth Frequent Items lists
[{'maggi'}, {'tea', 'maggi'}, {'bournvita'}, {'milk'}, {'cornflakes'}, {'cornflakes', 'coffee'}, {'suger'}, {'suger', 'coffee'}, {'tea'}, {'biscuit'}, {'coffee'}, {'bread'}]

FP Rules
[[{'tea'}, {'maggi'}, 0.5714285714285714], [{'maggi'}, {'tea'}, 0.8], [{'cornflakes'}, {'coffee'}, 0.6666666666666666], [{'coffee'}, {'cornflakes'}, 0.5], [{'suger'}, {'coffee'}, 0.6666666666666666], [{'coffee'}, {'suger'}, 0.5]]
